In [ ]:
import os
import time
import os, time, glob
import numpy as np
import pandas as pd
import librosa as lb
from tqdm import tqdm
import seaborn as sns
from scipy.io import loadmat
from scipy.signal.windows import gaussian
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn import metrics
from keras.regularizers import l2
from tensorflow.random import set_seed
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, GlobalAveragePooling2D, AvgPool2D  
from keras import Model, Input
from tensorflow.keras.optimizers import Adam
from keras.models import load_model

In [ ]:
# Load Dataset B

In [ ]:
BASE_PATH = '/kaggle/input/eeg-ad-mci-ctrl-csv/3-EEG_AD_MCI_CTRL_CSV'
sub_lst_all = sorted(glob.glob(BASE_PATH+'/*/*', recursive=True))
# sub_lst = sorted(glob.glob(BASE_PATH+'/CTRL/CTRL_*.csv', recursive=True))
# sub_lst = sorted(glob.glob(BASE_PATH+'/AD/AD_*.csv', recursive=True))
len(sub_lst_all)

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

X, y = [], []
segment_length = 1024

for count, sub in tqdm(enumerate(sub_lst_all)):
    signal = np.array(pd.read_csv(sub, header=None).T)
    
    filename = os.path.basename(os.path.normpath(sub))
    class_name = filename.split("_")[0]
#     print(len(signal[1]))
    # Calculate the number of segments
    num_segments = len(signal[1]) // segment_length
    
    # Extract segments and labels
    start, end = 0, segment_length
    for i in range(num_segments):
        start, end = i * segment_length, (i+1) * segment_length
        segment = signal[:, start:end]
        
        X.append(segment)
        y.append(class_name)

X = np.vstack(X)
y = np.array(y)
X = np.reshape(X,(-1,19,1024))

print('Input:', X.shape)
print('Output:', y.shape)

In [ ]:
unique_values, counts = np.unique(y, return_counts=True)

# Print the results
for value, count in zip(unique_values, counts):
    print(f"{value} occurs {count} times")

In [ ]:
# LOAD Dataset A

In [ ]:
BASE_PATH = '/kaggle/input/eeg-frontotemporal-dementia/3_EEG_Frontotemporal_Dementia_AD_CTRL/derivatives'
sub_lst = sorted(glob.glob(BASE_PATH+'/*/*', recursive=True))

df = pd.read_csv("/kaggle/input/eeg-frontotemporal-dementia/3_EEG_Frontotemporal_Dementia_AD_CTRL/participants.csv")
sub_lst = sorted(glob.glob(BASE_PATH+'/**/sub-*_task-eyesclosed_eeg.set', recursive=True))
labels = []
for sub in sub_lst:
    labels.extend(df[df["participant_id"] == sub.split('/')[-1].split('_')[0]]['Group'].to_list())
labels    

for lab in sorted(set(labels)):
    print('{:s} : {:d}'.format(lab, labels.count(lab)))

In [ ]:
X, y = [], []
segment_length = 1024

for count, sub in tqdm(enumerate(sub_lst)):
    signal = np.array(mne.io.read_raw(sub_lst[count], preload=True).to_data_frame().drop(columns=['time'], axis=1)).T

    class_name = labels[count]
#     print(len(signal[1]))
    # Calculate the number of segments
    num_segments = signal.shape[1] // segment_length
    
    # Extract segments and labels
    start, end = 0, segment_length
    for i in range(num_segments):
        start, end = i * segment_length, (i+1) * segment_length
        segment = signal[:, start:end]
        
        X.append(segment)
#         print(X)
        y.append(class_name)

X = np.array(X)
y = np.array(y)
X = np.reshape(X,(-1,19,1024))

print('Input:', X.shape)
print('Output:', y.shape)

In [ ]:
# Split Dataset

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0)

# y_train =np.array(y_train)
# y_test =np.array(y_test)
y_test_1 = np.array(y_test)

# scalar = StandardScaler()
# X_train = scalar.fit_transform(X_train.reshape(-1,1)).reshape(-1, 16, 1280)
# X_test  = scalar.transform(X_test.reshape(-1,1)).reshape(-1,16, 1280)

# X_train,y_train,X_test,y_test =SMOTE_Oversampling(X_train,X_test,y_train, y_test)
y_train =np.array(y_train)
y_test =np.array(y_test)
enc = OneHotEncoder()
y_train = enc.fit_transform(y_train.reshape(-1,1)).toarray()
y_test = enc.transform(y_test.reshape(-1,1)).toarray()

print('X_train: {0}\nX_test : {1}\ny_train: {2}\ny_test : {3}'.format(X_train.shape, X_test.shape, y_train.shape, y_test.shape))

In [ ]:
X_train =  X_train.reshape(-1, X_train.shape[1], X_train.shape[2],1)
X_test = X_test.reshape(-1, X_test.shape[1], X_test.shape[2],1)
print('X_train: {0}\nX_test : {1}'.format(X_train.shape, X_test.shape))

In [ ]:
input_shape = X_train[0,:].shape
print('Input shape: ',input_shape)

In [ ]:
import keras.backend as K
def spe(y_true, y_pred):
    true_negatives = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    possible_negatives = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    return true_negatives / (possible_negatives + K.epsilon())

def sen(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    return true_positives / (possible_positives + K.epsilon())

def Pre(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    return true_positives / (predicted_positives + K.epsilon())


def f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def mcc_metric(y_true, y_pred):
  predicted = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
  true_pos = tf.math.count_nonzero(predicted * y_true)
  true_neg = tf.math.count_nonzero((predicted - 1) * (y_true - 1))
  false_pos = tf.math.count_nonzero(predicted * (y_true - 1))
  false_neg = tf.math.count_nonzero((predicted - 1) * y_true)
  x = tf.cast((true_pos + false_pos) * (true_pos + false_neg) 
      * (true_neg + false_pos) * (true_neg + false_neg), tf.float32)
  return tf.cast((true_pos * true_neg) - (false_pos * false_neg), tf.float32) / tf.sqrt(x)

def fpr(y_true, y_pred):
#     predicted = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    false_pos = K.sum(K.round(K.clip(y_true, 0, 1)))
    true_neg = K.sum(K.round(K.clip((1 - y_true) * (1 - y_pred), 0, 1)))
    return (false_pos/(false_pos+true_neg))
    
def fnr(y_true, y_pred):
#     predicted = tf.cast(tf.greater(y_pred, 0.5), tf.float32)
    false_neg = K.sum(K.round(K.clip(1 - y_true, 0, 1)))
    true_pos = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    return (false_neg/(false_neg+true_pos))

In [ ]:
# create a Model

In [ ]:
input_shape = X_train[0,:].shape
model = Sequential([Conv2D(filters =10, kernel_size = (3,3), activation ='relu', input_shape = input_shape),
                    
                    MaxPool2D(pool_size=(2,21)), 
                    Dropout(0.25),
                    Conv2D(filters = 20,kernel_size = (3,3), activation ='relu', input_shape = input_shape,  kernel_regularizer=l2(0.02)),
                    
#                     GlobalAveragePooling2D(),
#                     AvgPool2D(),
                    Flatten(), 
#                     Dropout(0.5),
                    Dense(80, activation = "relu"),
#                     Dropout(0.25), 
                    Dense(50, activation = "relu"), 
                    Dense(3, activation = "softmax")])
model.summary()
start = time.time()
opt = Adam(learning_rate=0.0001)
model.compile(loss= 'binary_crossentropy', optimizer = opt ,metrics = ['accuracy',f1,sen,
                                                                       spe,mcc_metric,
                                                                       fpr, fnr])
                                                                       
history = model.fit(X_train, y_train, batch_size=80, epochs=100, shuffle=True,verbose=1,validation_split=0.2)

# print("Total time: {0:3.4f} sec".format(time.time() - start))

# score = model.evaluate(X_train, y_train, verbose=0)
# print("Training Accuracy: {0:3.4f}".format(score[1]))

score =model.evaluate(X_test, y_test, verbose=0)
print("Testing Accuracy: {0:3.4f}".format(score[1]))
# print("Testing loss: {0:3.4f}".format(score[0]))

# preds_test = model.predict(X_test) # label scores 
# preds_train = model.predict(X_train)
# print(confusion_matrix(y_test, y_pred))
# print('Train loss: {0:3.4f}'.format(score[0]))
# print('Train accuracy: {0:3.4f}'.format(score[1])
# model.save("ADNet.h5")

In [ ]:
# Performance parameter

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, matthews_corrcoef
preds_test = model.predict(X_test) # label scores 
# Make predictions
y_pred = np.argmax(preds_test, axis=1) # predicted classes 
y_test = np.argmax(y_test, axis=1) # true classes
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
conf_matrix = confusion_matrix(y_test, y_pred)

# Calculate sensitivity, specificity
sensitivity = recall_score(y_test, y_pred, average='macro')
specificity = np.diag(conf_matrix) / np.sum(conf_matrix, axis=1)

# Matthews correlation coefficient (MCC)
mcc = matthews_corrcoef(y_test, y_pred)

print("Accuracy:", accuracy)
print("F1 Score:", f1)
print("Precision:", precision)
print("Recall:", recall)
print("Sensitivity:", sensitivity)
print("Specificity:", specificity)
print("MCC:", mcc)
print("Confusion Matrix:\n", conf_matrix)

In [ ]:
# ROC Curve

In [ ]:
from sklearn.metrics import roc_curve, auc
from sklearn.preprocessing import label_binarize
from itertools import cycle
def plot_roc_curve(y_test, y_pred):
  
  n_classes = len(np.unique(y_test))
  y_test = label_binarize(y_test, classes=np.arange(n_classes))
  y_pred = label_binarize(y_pred, classes=np.arange(n_classes))

  # Compute ROC curve and ROC area for each class
  fpr = dict()
  tpr = dict()
  roc_auc = dict()
  for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_pred[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])
  
  # Compute micro-average ROC curve and ROC area
  fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_pred.ravel())
  roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

  # First aggregate all false positive rates
  all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

  # Then interpolate all ROC curves at this points
  mean_tpr = np.zeros_like(all_fpr)
  for i in range(n_classes):
    mean_tpr += np.interp(all_fpr, fpr[i], tpr[i])

  # Finally average it and compute AUC
  mean_tpr /= n_classes

  fpr["macro"] = all_fpr
  tpr["macro"] = mean_tpr
  roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])

  # Plot all ROC curves
  plt.figure(figsize=(5,5))
#   plt.figure(dpi=300)
  lw = 2
  plt.plot(fpr["micro"], tpr["micro"],
    label="micro-average (AUC = {0:0.2f})".format(roc_auc["micro"]),
    color="deeppink", linestyle=":", linewidth=4,)

  plt.plot(fpr["macro"], tpr["macro"],
    label="macro-average (AUC = {0:0.2f})".format(roc_auc["macro"]),
    color="navy", linestyle=":", linewidth=4,)
  class_name = ['AD', 'NC', 'FTD']
  colors = cycle(["aqua", "darkorange", "darkgreen", "yellow", "blue"])
  for i, color in zip(range(len(class_name)), colors):
    
        plt.plot(fpr[i], tpr[i], color=color, lw=lw,
                 
                
        label=" {0} (AUC = {1:0.2f})".format(class_name[i], roc_auc[i]))

  plt.plot([0, 1], [0, 1], "k--", lw=lw)
  plt.xlim([0.0, 1.0])
  plt.ylim([0.0, 1.05])
  plt.xlabel("False Positive Rate", weight='bold',fontsize=12)
  plt.ylabel("True Positive Rate", weight='bold',fontsize=12)
  plt.tick_params(labelsize=12)
#   plt.title("Receiver Operating Characteristic (ROC) curve")
#   fig.tight_layout(w_pad=2, h_pad=2 )
  
  plt.legend()
  plt.savefig('D2_roc_curv.pdf', dpi=300, bbox_inches="tight")

plot_roc_curve(y_test, y_pred)

In [ ]:
##Accuracy and Loss Cuve

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(9,3))
labels = ['Traning','Validation']
axs[0,].plot(history.history['accuracy'],'-r', lw=1.5)
axs[0].plot(history.history['val_accuracy'], '-k')
axs[0].set_xlabel('Number of epochs', weight='bold',fontsize=11)
axs[0].set_ylabel('Accuracy', weight='bold',fontsize=12)
axs[0].set_ylim([0.5, 1.05])
axs[0].tick_params(labelsize=11)
axs[0].legend( labels, loc='lower right',ncol=1)

axs[1].plot(history.history['loss'],'-r', lw=1.5)
axs[1].plot(history.history['val_loss'], '-k')
axs[1].set_xlabel('Number of epochs', weight='bold',fontsize=11)
axs[1].set_ylabel('Loss', weight='bold',fontsize=12)
# axs[1].set_ylim([0, 0.5])
axs[1].tick_params(labelsize=11)
axs[1].legend( labels, loc='upper right',ncol=1)

fig.tight_layout(w_pad=2, h_pad=2 )
plt.savefig('D2_LUCIDNet_loss.pdf')
plt.show()

In [ ]:
## FLIOPS calculation

In [ ]:
pip install keras-flops

import tensorflow as tf
from tensorflow.python.profiler.model_analyzer import profile
from tensorflow.python.profiler.option_builder import ProfileOptionBuilder

def get_flops(model):
  forward_pass = tf.function(model.call, input_signature=[tf.TensorSpec(shape=(1,) + model.input_shape[1:])])
  graph_info = profile(forward_pass.get_concrete_function().graph, options=ProfileOptionBuilder.float_operation())
  flops = graph_info.total_float_ops
  return flops

flops = get_flops(model)
macs = flops / 2
print(f"MACs: {macs / 1e+9:,} G")
print(f"FLOPs: {flops / 1e+9:,} G")
flops

In [ ]:
# Inference Time

In [ ]:
import time
import numpy as np

input_data = X_test  # Replace with your input shape
# model = VGG16()

start_time = time.time()
_ = model.predict(input_data)
end_time = time.time()

inference_time = end_time - start_time
print("Inference Time:", inference_time)

In [ ]:
# T-SNE plot of each layer

In [ ]:
import numpy as np

# Assuming your data is stored in variables X_test and y_test
# Replace these with your actual data
# X_test = np.random.random((6811, 19, 1024, 1))
# y_test = np.random.randint(0, 3, (6811,))

# Number of samples to select from each class
samples_per_class = 100

# Initialize an array to store the selected samples
selected_X = np.zeros((3 * samples_per_class, *X_test.shape[1:]))
selected_y = np.zeros(3 * samples_per_class, dtype=int)  # Use integer array

# Iterate through each class
for class_index in range(3):
    # Get indices of samples for the current class
    class_indices = np.where(y_test == class_index)[0]

    # Randomly select 'samples_per_class' samples for the current class
    selected_indices = np.random.choice(class_indices, samples_per_class, replace=False)

    # Copy the selected samples to the new arrays
    selected_X[class_index * samples_per_class: (class_index + 1) * samples_per_class] = X_test[selected_indices]
    selected_y[class_index * samples_per_class: (class_index + 1) * samples_per_class] = y_test[selected_indices]


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn import preprocessing
import tensorflow as tf
from tensorflow.keras.models import Model

# Assuming you have the intermediate representations for each layer
# Replace these with your actual data
successive_outputs = [layer.output for layer in model.layers]

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[0])
successive_feature_maps_layer0 = visualization_model.predict(selected_X)
layer0_data = np.array(successive_feature_maps_layer0)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[1])
successive_feature_maps_layer1 = visualization_model.predict(selected_X)
layer1_data = np.array(successive_feature_maps_layer1)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[2])
successive_feature_maps_layer2 = visualization_model.predict(selected_X)
layer2_data = np.array(successive_feature_maps_layer2)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[3])
successive_feature_maps_layer3 = visualization_model.predict(selected_X)
layer3_data = np.array(successive_feature_maps_layer3)

successive_outputs = [layer.output for layer in model.layers]
visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[4])
successive_feature_maps_layer4 = visualization_model.predict(selected_X)
layer4_data = np.array(successive_feature_maps_layer4)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[5])
successive_feature_maps_layer5 = visualization_model.predict(selected_X)
layer5_data = np.array(successive_feature_maps_layer5)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[6])
successive_feature_maps_layer6 = visualization_model.predict(selected_X)
layer6_data = np.array(successive_feature_maps_layer6)

visualization_model = tf.keras.models.Model(inputs=model.input, outputs=successive_outputs[7])
successive_feature_maps_layer7 = visualization_model.predict(selected_X)
layer7_data = np.array(successive_feature_maps_layer7)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE

def plot_and_save_tsne(layer_index, layer_data, labels, markers, save_path):
    # Define t-SNE model
    tsne = TSNE(n_components=2, random_state=42)

    # Reshape data before applying t-SNE
    tsne_data = tsne.fit_transform(layer_data.reshape(layer_data.shape[0], -1))

    plt.figure(figsize=(7, 4))
       
    # Plot t-SNE visualization for each class with different colors and markers
    for class_label in range(3):
        indices = labels == class_label
        plt.scatter(tsne_data[indices, 0], tsne_data[indices, 1], label=f'Class {class_label}',color=colors[class_label], alpha=1, marker=markers[class_label])
        plt.tick_params(labelsize=14)
        plt.locator_params(axis='x', nbins=5)
        plt.locator_params(axis='y', nbins=5)
#     plt.legend()
#     plt.title(f'T-SNE Visualization - Layer {layer_index}')
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()

# Assuming you have markers defined as in your code
markers = ['o', 's', '^']
colors = ['red', 'green', 'blue']

label_encoder = preprocessing.LabelEncoder()
label = label_encoder.fit_transform(selected_y)
labels = np.array(label)

# Call the function for each layer
plot_and_save_tsne(0, layer0_data, labels, markers, 'AD_FTD_NC_layer0_Tsne_plot.pdf')
plot_and_save_tsne(1, layer1_data, labels, markers, 'AD_FTD_NC_layer1_Tsne_plot.pdf')
plot_and_save_tsne(2, layer2_data, labels, markers, 'AD_FTD_NC_layer2_Tsne_plot.pdf')
plot_and_save_tsne(3, layer3_data, labels, markers, 'AD_FTD_NC_layer3_Tsne_plot.pdf')
plot_and_save_tsne(4, layer4_data, labels, markers, 'AD_FTD_NC_layer4_Tsne_plot.pdf')
plot_and_save_tsne(5, layer5_data, labels, markers, 'AD_FTD_NC_layer5_Tsne_plot.pdf')
plot_and_save_tsne(6, layer6_data, labels, markers, 'AD_FTD_NC_layer6_Tsne_plot.pdf')
plot_and_save_tsne(7, layer7_data, labels, markers, 'AD_FTD_NC_layer7_Tsne_plot.pdf')